In [ ]:
!pip install optuna
!pip install matplotlib
!pip install seaborn
!pip install lightgbm
!pip install xgboost
!pip install scikit-learn
!pip install catboost

In [3]:
# df = pd.read_csv('/opt/ml/input/data/train_dataset/train_time_finalfix.csv')


# # df['Timestamp']=pd.to_datetime(df['Timestamp'].values)
# # df['month'] = df['Timestamp'].dt.month
# # # df['userID'] = (df['userID'].map(str)+df['month'].apply(lambda x: ('0'+str(x)) if x<10 else str(x))).astype('int32')

# # df['userID'] = df['userID'].map(str)+'-'+df['month'].map(str)
# # df.drop(columns=['month'],inplace=True)
# print("user_augmentation 전 유저 수",len(df['userID'].unique()))


user_augmentation 전 유저 수 6698


In [ ]:
# df['Timestamp']=pd.to_datetime(df['Timestamp'].values)
# df['month'] = df['Timestamp'].dt.month
# # df['userID'] = (df['userID'].map(str)+df['month'].apply(lambda x: ('0'+str(x)) if x<10 else str(x))).astype('int32')

# df['userID'] = df['userID'].map(str)+'-'+df['month'].map(str)
# df.drop(columns=['month'],inplace=True)
# print("user_augmentation 후 유저 수",len(df['userID'].unique()))


In [2]:
import numpy as np
import pandas as pd
import os
import random
from collections import defaultdict
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)
from tqdm.notebook import tqdm
   
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool  
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

In [4]:
df1 = pd.read_csv('/opt/ml/input/data/train_dataset/cw_train.csv') 

In [5]:
# LOAD TESTDATA
test_df = pd.read_csv('/opt/ml/input/data/train_dataset/JH_test_v3.csv')

In [6]:
# LOAD TESTDATA
test_df1 = pd.read_csv('/opt/ml/input/data/train_dataset/cw_test.csv')

In [7]:
df = df[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'test_level_diff_itemID',
       'test_level_diff_KnowledgeTag', 'test_level_diff_testId',
       'user_correct_answer_tag', 'user_total_answer_tag', 'user_acc_tag',
       'sec_time', 'solve_time', 'test_t_mean', 'test_t_std', 'test_t_skew',
       'assess_t_mean', 'assess_t_std', 'assess_t_skew', 'tag_t_mean',
       'tag_t_std', 'tag_t_skew', 'test_level', 'item_ans_Rate']]
df1 = df1[['userID', 'assessmentItemID', 'testId','Timestamp','user_correct_answer',
       'user_total_answer', 'user_acc', 'u_test_cnt', 'test_ans_cnt',
       'test_ans_rate', 'u_tag_cnt', 'tag_ans_cnt', 'tag_ans_rate',
       'test_rate', 'grade', 'grade_test_level', 'que_rate', 'que_num',
       'que_num_rate', 'tag_rate', 'user_grade']]
df = pd.merge(df,df1,on = ['userID','assessmentItemID','testId','Timestamp'],how = 'left')

In [8]:

df.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_level_diff_itemID,test_level_diff_KnowledgeTag,test_level_diff_testId,user_correct_answer_tag,...,tag_ans_cnt,tag_ans_rate,test_rate,grade,grade_test_level,que_rate,que_num,que_num_rate,tag_rate,user_grade
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.384,0.400000,0.378667,NaN,...,0.0,0.5,0.95,6,1,0.98,1,0.75,0.96,8
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0.376,0.400533,0.378667,NaN,...,0.0,0.5,0.95,6,1,0.97,2,0.72,0.92,8
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0.376,0.400533,0.378667,1.0,...,1.0,1.0,0.95,6,1,0.92,3,0.69,0.92,8
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0.392,0.400533,0.378667,2.0,...,2.0,1.0,0.95,6,1,0.97,4,0.66,0.92,8
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0.376,0.400533,0.378667,3.0,...,3.0,1.0,0.95,6,1,0.95,5,0.60,0.92,8


In [9]:
test_df = test_df[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'test_level_diff_itemID',
       'test_level_diff_KnowledgeTag', 'test_level_diff_testId', 
       'user_correct_answer_tag', 'user_total_answer_tag', 'user_acc_tag',
       'sec_time', 'solve_time', 'test_t_mean', 'test_t_std', 'test_t_skew',
       'assess_t_mean', 'assess_t_std', 'assess_t_skew', 'tag_t_mean',
       'tag_t_std', 'tag_t_skew', 'test_level', 'item_ans_Rate']]
test_df1 = test_df1[['userID', 'assessmentItemID', 'testId','Timestamp','user_correct_answer',
       'user_total_answer', 'user_acc', 'u_test_cnt', 'test_ans_cnt',
       'test_ans_rate', 'u_tag_cnt', 'tag_ans_cnt', 'tag_ans_rate',
       'test_rate', 'grade', 'grade_test_level', 'que_rate', 'que_num',
       'que_num_rate', 'tag_rate']]
test_df = pd.merge(test_df,test_df1,on = ['userID','assessmentItemID','testId','Timestamp'],how = 'left')

In [10]:
test_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_level_diff_itemID,test_level_diff_KnowledgeTag,test_level_diff_testId,user_correct_answer_tag,...,u_tag_cnt,tag_ans_cnt,tag_ans_rate,test_rate,grade,grade_test_level,que_rate,que_num,que_num_rate,tag_rate
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.192000,0.214857,0.197714,NaN,...,0,0.0,0.5,0.55,5,34,0.66,1,0.75,0.64
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,0.200000,0.214857,0.197714,1.0,...,1,1.0,1.0,0.55,5,34,0.62,2,0.72,0.64
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,0.216000,0.232000,0.197714,NaN,...,0,0.0,0.5,0.55,5,34,0.55,3,0.69,0.66
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.176000,0.232000,0.197714,0.0,...,1,0.0,0.0,0.55,5,34,0.64,4,0.66,0.66
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.152000,0.222222,0.197714,NaN,...,0,0.0,0.5,0.55,5,34,0.30,6,0.56,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.193333,0.229715,0.245497,NaN,...,0,0.0,0.5,0.64,4,30,0.44,1,0.75,0.64
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,0.193333,0.229715,0.245497,0.0,...,1,0.0,0.0,0.64,4,30,0.48,2,0.72,0.64
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,0.293333,0.267112,0.245497,NaN,...,0,0.0,0.5,0.64,4,30,0.85,3,0.69,0.77
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,0.286667,0.267112,0.245497,1.0,...,1,1.0,1.0,0.64,4,30,0.83,4,0.66,0.77


In [11]:
model_name = 'catboost'

In [12]:
import os,gc
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve, accuracy_score

#폴드 개수만큼 user를 쪼갠 목록을 만든다
def lgbm_oof_split_data_withidx(df, n_fold):
    #grade별로 user list 생성 (9개)
    u_g = df.groupby(['userID','user_grade'])['userID'].count().reset_index(name='cnt')   #user별 grade값
    users = [[] for _ in range(10)]  #grade개수총 9개
    for id in u_g['userID']:
        id_grade = int(u_g[u_g['userID'] == id]['user_grade'])  #해당 유저의 grade
        id_cnt = int(u_g[u_g['userID'] == id]['cnt'])
        users[id_grade].append((id, id_cnt))
    # grade별로 sorted
    for i in range(1,10):
        user_list = users[i]
        user_list = sorted(user_list, key =lambda x: x[1], reverse=True)
        users[i] = user_list
        
    n_fold = n_fold
    random.seed(42)
    user_id_dict=defaultdict(list)
    user_count_dict=defaultdict(int)
    for i in range(1,10):
        gul = users[i]           # gul = grade user list
        random.shuffle(gul)
        for idx, (id, cnt) in enumerate(gul):
            f_num=idx%n_fold
            user_count_dict[f_num] += cnt
            user_id_dict[f_num].append(id)
    return user_id_dict, user_count_dict

In [13]:
#dict들을 통해 원하는 폴드의 train/val set을 가져온다
def get_fold_data(idx, data, user_id_dict,user_count_dict, train_user_id):
    train = data[data['userID'].isin(user_id_dict[idx]) == False]  # train
    valid = data[data['userID'].isin(user_id_dict[idx])] # validation
    
    #test데이터셋은 각 유저의 마지막 interaction만 추출
    df_last = valid['userID'] != valid['userID'].shift(-1)  #validation
    train_UID = valid['userID'].isin(train_user_id)

    oof_valid = valid[df_last & train_UID].copy()
    
    valid.drop(valid[df_last & train_UID].index, inplace=True)
    
    train = pd.concat([train, valid])
    
    return train, oof_valid

In [14]:
#FE를 진행한 전체 train set과 FE를 진행한 전체 test set, featrue 들을 넣어주시면 됩니다
def make_lgb_user_oof_prediction(train, FEATS, train_user_id, test, model_params=None, folds=5,model_name = 'lgbm'):  # categorical_features='auto',
#     train.shape[0]  # train 행 총 개수
    # fold 생성
    user_id_dict,user_count_dict = lgbm_oof_split_data_withidx(train, folds)
    
    # test data
    x_test = test[FEATS] # 마지막 interaction의 features 값
    
    
    # 테스트 데이터 예측값을 저장할 변수
    test_preds = np.zeros(x_test.shape[0])
    test_df = test['userID'].to_frame().copy()
    
    # Out Of Fold Validation 예측 데이터를 저장할 변수 (예측값)
    oof_df = pd.DataFrame()
    
    # 폴드별 평균 Validation 스코어를 저장할 변수
    score = 0
    acc=0
    
    # 피처 중요도를 저장할 데이터 프레임 선언
    fi = pd.DataFrame()
    fi['feature'] = FEATS
    
    if model_name =='xgb':
      x_test = xgb.DMatrix(x_test)

    for fold in range(folds):
        # train index, validation index로 train 데이터를 나눔
        train_set, valid_set= get_fold_data(fold, train, user_id_dict, user_count_dict, train_user_id)  #개수 잘 나뉘었다.
        
        x_tr, x_val = train_set[FEATS], valid_set[FEATS]    # 정답 외 feature 값들 (마지막 interaction만 X)
        y_tr, y_val = train_set['answerCode'], valid_set['answerCode']  # 정답 부분
        print(f'fold: {fold+1}, x_tr.shape: {x_tr.shape}, x_val.shape: {x_val.shape}')
        
        # LightGBM 
        if model_name =='lgbm':
            dtrain = lgb.Dataset(x_tr, label=y_tr)
            dvalid = lgb.Dataset(x_val, label=y_val)
        
        # LightGBM 모델 훈련
            model = lgb.train(
                model_params,
                dtrain,
                valid_sets=[dtrain, dvalid], # Validation 성능을 측정할 수 있도록 설정
                categorical_feature='auto',
                verbose_eval=100,    
                num_boost_round=5000,
                early_stopping_rounds=100    
            )

            # Validation 데이터 예측
            val_preds = model.predict(x_val)
            
        elif model_name =='catboost':
            model = CatBoostClassifier(**model_params)
            model.fit(x_tr, y_tr,
                    eval_set=(x_val, y_val), # Validation 성능을 측정할 수 있도록 설정
#                     cat_features=categorical_features,
                    use_best_model=True,
                    verbose=True)

            # Validation 데이터 예측
            val_preds = model.predict_proba(x_val)[:,1]
            
        elif model_name =='xgb':
            dtrain = xgb.DMatrix(x_tr, label=y_tr)
            dvalid = xgb.DMatrix(x_val, label=y_val)

            # XGBoost 모델 훈련
            model = xgb.train(
                model_params,
                dtrain,
                num_boost_round=10000, # 트리 개수
                evals=[(dtrain, 'train'), (dvalid, 'valid')],  # Validation 성능을 측정할 수 있도록 설정
                verbose_eval=100,
                early_stopping_rounds=100
            )

            # Validation 데이터 예측
            val_preds = model.predict(dvalid)
            
                        
        # oof validation 예측값 저장 
        valid_set['preds'] = val_preds
        
        oof_df = pd.concat([oof_df, valid_set[['userID', 'preds','answerCode']]])

        # 폴드별 Validation 스코어 측정
        fold_acc = accuracy_score(y_val, list(map(round,val_preds)))
        print(f"Fold {fold + 1} | AUC: {roc_auc_score(y_val, val_preds)} | ACC: {fold_acc}")
        print('-'*80)
        
        # score 변수에 폴드별 평균 Validation 스코어 저장
        score += roc_auc_score(y_val, val_preds) / folds
        acc+=fold_acc / folds
        
        # 테스트 데이터 예측하고 평균해서 저장
        test_preds += model.predict(x_test) / folds

        
        # 폴드별 피처 중요도 저장
        # fi[f'fold_{fold+1}'] = model.feature_importance()

        del x_tr, x_val, y_tr, y_val
        gc.collect()
        
    print(f"\nMean AUC = {score}") # 폴드별 Validation 스코어 출력

    # 폴드별 피처 중요도 평균값 계산해서 저장 
    # fi_cols = [col for col in fi.columns if 'fold_' in col]
    # fi['importance'] = fi[fi_cols].mean(axis=1)
    
    # fi = fi.sort_values('importance', ascending = False).reset_index(drop = True)
    
    # 피처 중요도 정규화 및 누적 중요도 계산
    # fi['importance_normalized'] = fi['importance'] / fi['importance'].sum()
    # fi['cumulative_importance'] = np.cumsum(fi['importance_normalized'])

    # make test_df
    test_df['preds'] = test_preds
    
    return oof_df, test_df,fi

In [15]:
model_params = {        
            'objective': 'binary:logistic', 
            'eval_metric': 'auc'}

In [16]:
df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'test_level_diff_itemID',
       'test_level_diff_KnowledgeTag', 'test_level_diff_testId',
       'user_correct_answer_tag', 'user_total_answer_tag', 'user_acc_tag',
       'sec_time', 'solve_time', 'test_t_mean', 'test_t_std', 'test_t_skew',
       'assess_t_mean', 'assess_t_std', 'assess_t_skew', 'tag_t_mean',
       'tag_t_std', 'tag_t_skew', 'test_level', 'item_ans_Rate',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'u_test_cnt',
       'test_ans_cnt', 'test_ans_rate', 'u_tag_cnt', 'tag_ans_cnt',
       'tag_ans_rate', 'test_rate', 'grade', 'grade_test_level', 'que_rate',
       'que_num', 'que_num_rate', 'tag_rate', 'user_grade'],
      dtype='object')

In [17]:
FEATS = ['test_level_diff_itemID',
       'test_level_diff_KnowledgeTag', 'test_level_diff_testId', 
       'user_correct_answer_tag', 'user_total_answer_tag', 'user_acc_tag',
       'sec_time', 'solve_time', 'test_t_mean', 'test_t_std', 'test_t_skew',
       'assess_t_mean', 'assess_t_std', 'assess_t_skew', 'tag_t_mean',
       'tag_t_std', 'tag_t_skew', 'test_level', 'item_ans_Rate',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'u_test_cnt',
       'test_ans_cnt', 'test_ans_rate', 'u_tag_cnt', 'tag_ans_cnt',
       'tag_ans_rate', 'test_rate', 'grade', 'grade_test_level', 'que_rate',
       'que_num', 'que_num_rate', 'tag_rate'] 

In [18]:
test_df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_level_diff_itemID,test_level_diff_KnowledgeTag,test_level_diff_testId,user_correct_answer_tag,...,u_tag_cnt,tag_ans_cnt,tag_ans_rate,test_rate,grade,grade_test_level,que_rate,que_num,que_num_rate,tag_rate
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.192000,0.214857,0.197714,NaN,...,0,0.0,0.5,0.55,5,34,0.66,1,0.75,0.64
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,0.200000,0.214857,0.197714,1.0,...,1,1.0,1.0,0.55,5,34,0.62,2,0.72,0.64
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,0.216000,0.232000,0.197714,NaN,...,0,0.0,0.5,0.55,5,34,0.55,3,0.69,0.66
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,0.176000,0.232000,0.197714,0.0,...,1,0.0,0.0,0.55,5,34,0.64,4,0.66,0.66
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,0.152000,0.222222,0.197714,NaN,...,0,0.0,0.5,0.55,5,34,0.30,6,0.56,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.193333,0.229715,0.245497,NaN,...,0,0.0,0.5,0.64,4,30,0.44,1,0.75,0.64
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,0.193333,0.229715,0.245497,0.0,...,1,0.0,0.0,0.64,4,30,0.48,2,0.72,0.64
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,0.293333,0.267112,0.245497,NaN,...,0,0.0,0.5,0.64,4,30,0.85,3,0.69,0.77
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,0.286667,0.267112,0.245497,1.0,...,1,1.0,1.0,0.64,4,30,0.83,4,0.66,0.77


In [19]:
# LEAVE LAST INTERACTION ONLY
test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]
# DROP ANSWERCODE
test_df = test_df.drop(['answerCode'], axis=1)
test_df.head()

,userID,assessmentItemID,testId,Timestamp,KnowledgeTag,test_level_diff_itemID,test_level_diff_KnowledgeTag,test_level_diff_testId,user_correct_answer_tag,user_total_answer_tag,...,u_tag_cnt,tag_ans_cnt,tag_ans_rate,test_rate,grade,grade_test_level,que_rate,que_num,que_num_rate,tag_rate
1035,3,A050133008,A050000133,2020-10-26 13:13:57,5289,0.265060,0.244383,0.283142,9.0,11,...,11,9.0,0.82,0.66,5,23,0.53,8,0.46,0.56
1706,4,A070146008,A070000146,2020-12-27 02:47:54,9080,0.234483,0.181747,0.217573,2.0,3,...,3,2.0,0.67,0.65,7,9,0.61,8,0.46,0.55
3023,13,A070111008,A070000111,2020-12-27 04:35:09,9660,0.112450,0.136849,0.110055,2.0,6,...,6,2.0,0.33,0.46,7,28,0.37,8,0.46,0.49
4283,17,A090064006,A090000064,2020-10-30 05:48:37,2611,0.121212,0.141343,0.136895,5.0,5,...,5,5.0,1.00,0.45,9,13,0.27,6,0.56,0.42
4670,26,A060135007,A060000135,2020-10-23 11:44:18,1422,0.208835,0.267004,0.311035,4.0,6,...,6,4.0,0.67,0.64,6,29,0.31,7,0.52,0.61


In [20]:
train_user_id = df['userID'].unique()

In [21]:
y_oof, test_preds, fi = make_lgb_user_oof_prediction(df, FEATS, train_user_id, test_df, model_params=model_params, folds=5,model_name = model_name)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 1, x_tr.shape: (2524463, 35), x_val.shape: (1493, 35)
[0]	train-auc:0.82345	valid-auc:0.79873
[100]	train-auc:0.85472	valid-auc:0.83258
[200]	train-auc:0.85853	valid-auc:0.83396
[300]	train-auc:0.86173	valid-auc:0.83458
[400]	train-auc:0.86442	valid-auc:0.83495


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 2, x_tr.shape: (2524465, 35), x_val.shape: (1491, 35)
[0]	train-auc:0.82346	valid-auc:0.79830
[100]	train-auc:0.85459	valid-auc:0.83065
[200]	train-auc:0.85852	valid-auc:0.83203
[300]	train-auc:0.86165	valid-auc:0.83080
[334]	train-auc:0.86257	valid-auc:0.83122
Fold 2 | AUC: 0.8312177899979847 | ACC: 0.7518443997317237
--------------------------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 3, x_tr.shape: (2524467, 35), x_val.shape: (1489, 35)
[0]	train-auc:0.82345	valid-auc:0.80994
[100]	train-auc:0.85467	valid-auc:0.84345
[200]	train-auc:0.85866	valid-auc:0.84447
[258]	train-auc:0.86052	valid-auc:0.84484
Fold 3 | AUC: 0.845023857475944 | ACC: 0.770987239758227
--------------------------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 4, x_tr.shape: (2524471, 35), x_val.shape: (1485, 35)
[0]	train-auc:0.82345	valid-auc:0.80469
[100]	train-auc:0.85458	valid-auc:0.83329
[200]	train-auc:0.85853	valid-auc:0.83431
[229]	train-auc:0.85949	valid-auc:0.83479
Fold 4 | AUC: 0.8347903719084903 | ACC: 0.7508417508417509
--------------------------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 5, x_tr.shape: (2524472, 35), x_val.shape: (1484, 35)
[0]	train-auc:0.82367	valid-auc:0.80560
[100]	train-auc:0.85460	valid-auc:0.83099
[200]	train-auc:0.85837	valid-auc:0.83204
[300]	train-auc:0.86144	valid-auc:0.83307
[400]	train-auc:0.86417	valid-auc:0.83327
[455]	train-auc:0.86560	valid-auc:0.83307
Fold 5 | AUC: 0.8330680562823419 | ACC: 0.7580862533692723
--------------------------------------------------------------------------------

Mean AUC = 0.8357404597115945


In [24]:

def objective(trial,model_name = model_name):    
    if model_name == 'lgbm':
        model_params = {
            'objective': 'binary', # 이진 분류
            'boosting_type': 'gbdt',
            'metric':'auc',
            'num_leaves': trial.suggest_int('num_leaves', 2, 256), # num_leaves 값을 2-256까지 정수값 중에 사용
            'max_bin': trial.suggest_int('max_bin', 128, 256), # max_bin 값을 128-256까지 정수값 중에 사용
            # min_data_in_leaf 값을 10-40까지 정수값 중에 사용
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 40),
            # 피처 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            # 데이터 샘플링 비율을 0.4-1.0까지 중에 uniform 분포로 사용
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            # 데이터 샘플링 횟수를 1-7까지 정수값 중에 사용
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#             'n_estimators': 10000, # 트리 개수
#             'early_stopping_rounds': 100,
            # L1 값을 1e-8-10.0까지 로그 uniform 분포로 사용
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
            # L2 값을 1e-8-10.0까지 로그 uniform 분포로 사용
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
            'seed': 42,
            'verbose': -1,
            'n_jobs': -1,    
        }
    elif model_name == 'catboost':
        model_params = {
            'n_estimators': 10000,
            'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
            'eval_metric': 'AUC', # 평가 지표 설정
            'loss_function': 'Logloss', # 손실 함수 설정
            'random_seed': 42,
            'metric_period': 100,
            'od_wait': 100, # early stopping round
            'depth' : trial.suggest_int('depth', 4, 10),
            'rsm': trial.suggest_uniform('rsm', 0.4, 1.0)        
        }
    
    elif model_name == 'xgb':
        model_params = {        
            'objective': 'binary:logistic', 
            'eval_metric': 'auc',
            'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),
            'max_depth': trial.suggest_int("max_depth", 3, 9),        
            'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
            'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
            "subsample": trial.suggest_float("subsample", 0.2, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 2, 10)
        }
    y_oof, test_preds, fi = make_lgb_user_oof_prediction(df, FEATS, train_user_id, test_df, model_params=model_params, folds=5,model_name = model_name )    
           
    
#     acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_oof['answerCode'], y_oof['preds'])
    return auc

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10) # 10회 동안 하이퍼 파라미터 탐색
study.best_params

[I 2021-06-14 06:23:42,632] A new study created in memory with name: no-name-5da50e71-5736-4e74-9d0f-97789bc009dd
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 1, x_tr.shape: (2524463, 35), x_val.shape: (1493, 35)
[0]	train-auc:0.82347	valid-auc:0.79744
[100]	train-auc:0.83880	valid-auc:0.81515
[200]	train-auc:0.84429	valid-auc:0.82001
[300]	train-auc:0.84741	valid-auc:0.82292
[400]	train-auc:0.84908	valid-auc:0.82463
[500]	train-auc:0.85006	valid-auc:0.82576
[600]	train-auc:0.85078	valid-auc:0.82659
[700]	train-auc:0.85134	valid-auc:0.82729
[800]	train-auc:0.85182	valid-auc:0.82772
[900]	train-auc:0.85228	valid-auc:0.82826
[1000]	train-auc:0.85267	valid-auc:0.82887
[1100]	train-auc:0.85302	valid-auc:0.82928
[1200]	train-auc:0.85337	valid-auc:0.82964
[1300]	train-auc:0.85368	valid-auc:0.82988
[1400]	train-auc:0.85398	valid-auc:0.83022
[1500]	train-auc:0.85425	valid-auc:0.83052
[1600]	train-auc:0.85450	valid-auc:0.83091
[1700]	train-auc:0.85478	valid-auc:0.83107
[1800]	train-auc:0.85503	valid-auc:0.83134
[1900]	train-auc:0.85526	valid-auc:0.83142
[2000]	train-auc:0.85546	valid-auc:0.83165
[2100]	train-auc:0.85570	valid-auc:0.83172
[2200]

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 2, x_tr.shape: (2524465, 35), x_val.shape: (1491, 35)
[0]	train-auc:0.82275	valid-auc:0.79926
[100]	train-auc:0.83855	valid-auc:0.81587
[200]	train-auc:0.84423	valid-auc:0.81969
[300]	train-auc:0.84740	valid-auc:0.82273
[400]	train-auc:0.84905	valid-auc:0.82473
[500]	train-auc:0.85008	valid-auc:0.82579
[600]	train-auc:0.85081	valid-auc:0.82647
[700]	train-auc:0.85144	valid-auc:0.82719
[800]	train-auc:0.85190	valid-auc:0.82768
[900]	train-auc:0.85231	valid-auc:0.82793
[1000]	train-auc:0.85267	valid-auc:0.82798
[1100]	train-auc:0.85302	valid-auc:0.82827
[1200]	train-auc:0.85334	valid-auc:0.82844
[1300]	train-auc:0.85365	valid-auc:0.82869
[1400]	train-auc:0.85394	valid-auc:0.82871
[1500]	train-auc:0.85422	valid-auc:0.82891
[1577]	train-auc:0.85443	valid-auc:0.82889
Fold 2 | AUC: 0.8289145049376673 | ACC: 0.7491616364855801
--------------------------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 3, x_tr.shape: (2524467, 35), x_val.shape: (1489, 35)
[0]	train-auc:0.82280	valid-auc:0.80837
[100]	train-auc:0.83889	valid-auc:0.82693
[200]	train-auc:0.84437	valid-auc:0.83247
[300]	train-auc:0.84745	valid-auc:0.83596
[400]	train-auc:0.84915	valid-auc:0.83787
[500]	train-auc:0.85020	valid-auc:0.83952
[600]	train-auc:0.85094	valid-auc:0.84044
[700]	train-auc:0.85152	valid-auc:0.84111
[800]	train-auc:0.85202	valid-auc:0.84179
[900]	train-auc:0.85237	valid-auc:0.84215
[1000]	train-auc:0.85273	valid-auc:0.84254
[1100]	train-auc:0.85308	valid-auc:0.84283
[1200]	train-auc:0.85344	valid-auc:0.84327
[1300]	train-auc:0.85373	valid-auc:0.84346
[1400]	train-auc:0.85399	valid-auc:0.84360
[1500]	train-auc:0.85425	valid-auc:0.84377
[1600]	train-auc:0.85452	valid-auc:0.84397
[1700]	train-auc:0.85477	valid-auc:0.84405
[1800]	train-auc:0.85503	valid-auc:0.84420
[1900]	train-auc:0.85527	valid-auc:0.84426
[1991]	train-auc:0.85549	valid-auc:0.84422
Fold 3 | AUC: 0.844222592777068 | ACC: 0.77031564

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


fold: 4, x_tr.shape: (2524471, 35), x_val.shape: (1485, 35)
[0]	train-auc:0.82339	valid-auc:0.80478
[100]	train-auc:0.83899	valid-auc:0.81675
[200]	train-auc:0.84431	valid-auc:0.82193
[300]	train-auc:0.84751	valid-auc:0.82484
[400]	train-auc:0.84908	valid-auc:0.82694
[500]	train-auc:0.85009	valid-auc:0.82842
[600]	train-auc:0.85081	valid-auc:0.82960
[700]	train-auc:0.85135	valid-auc:0.83049


In [ ]:
y_oof.to_csv('./valid_preds.csv', index=False)
test_preds.to_csv('./test_preds.csv', index=False)
test_preds.reset_index(inplace=True)
test_preds.reset_index(inplace=True)
del test_preds['userID']
del test_preds['index']
test_preds.rename(columns = {'level_0' : 'id','preds':'prediction'}, inplace = True)
test_preds.to_csv('./submission.csv',index=False)

In [ ]:
test_preds